## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
print (tf.__version__)

2.0.0


In [3]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [4]:
"""
from google.colab import drive
drive.mount('/content/drive/')
"""

"\nfrom google.colab import drive\ndrive.mount('/content/drive/')\n"

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv("C:/Users/Parikshit/Desktop/great lakes course/Week 6 Assignment/Internal lab/prices.csv")

### Check all columns in the dataset

In [7]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [8]:
data.drop(columns=['date','symbol'],axis=1,inplace=True)

In [9]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [10]:
data = data[:1000]

In [11]:
x = data.iloc[:,0:4]

In [12]:
y = data.iloc[:,4:]/1000000
y

,volume
0,2.1636
1,2.3864
2,2.4895
3,2.0063
4,1.4086
...,...
995,2.1332
996,1.9824
997,37.1528
998,6.5686


### Divide the data into train and test sets

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.20,random_state=3)

#### Convert Training and Test Data to numpy float32 arrays


In [14]:
import numpy as np
X_train =np.array(X_train).astype('float32')
X_test = np.array(X_test).astype('float32')
Y_train =np.array(Y_train).astype('float32')
Y_test = np.array(Y_test).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [15]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [16]:
W = tf.zeros(shape=(4, 1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [17]:

def prediction(X_train, W, b):
    y_pred = tf.add(tf.matmul(X_train, W), b)
    return y_pred

3.Loss (Cost) Function [Mean square error]

In [18]:

def loss(Y_train, y_predicted):
    diff = Y_train - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [19]:

def train(X_train, Y_train, w, b, learning_rate=0.01):
    
    # Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([w,b])
        current_prediction = prediction(X_train, w, b)
        current_loss = loss(Y_train, current_prediction)
    
    # Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    # Update Weights and Bias
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [20]:
for i in range(100):    
    W, b = train(X_train, Y_train, W, b)
    print('Current Training Loss on iteration', i, ' -- ', loss(Y_train, prediction(X_train, W, b)).numpy())

InternalError: Blas GEMV launch failed:  m=4, n=800 [Op:MatMul] name: MatMul/

In [58]:
for i in range(100):    
    W, b = train(X_train, Y_train, W, b)
    if i%5 == 0:
      print('Current Training Loss on iteration : ', i,' -- ', loss(Y_train, prediction(X_train, W, b)).numpy())
    

Current Training Loss on iteration :  0  --  172.03566
Current Training Loss on iteration :  5  --  172.03319
Current Training Loss on iteration :  10  --  172.03151
Current Training Loss on iteration :  15  --  172.0304
Current Training Loss on iteration :  20  --  172.02963
Current Training Loss on iteration :  25  --  172.0291
Current Training Loss on iteration :  30  --  172.02875
Current Training Loss on iteration :  35  --  172.0285
Current Training Loss on iteration :  40  --  172.02832
Current Training Loss on iteration :  45  --  172.02818
Current Training Loss on iteration :  50  --  172.02809
Current Training Loss on iteration :  55  --  172.02803
Current Training Loss on iteration :  60  --  172.02792
Current Training Loss on iteration :  65  --  172.02791
Current Training Loss on iteration :  70  --  172.02785
Current Training Loss on iteration :  75  --  172.02779
Current Training Loss on iteration :  80  --  172.02777
Current Training Loss on iteration :  85  --  172.027

### Get the shapes and values of W and b

In [59]:
print ('shape of W : ', W)

shape of W :  tf.Tensor(
[[1.2862535]
 [1.3554296]
 [1.2863495]
 [1.3583084]], shape=(4, 1), dtype=float32)


In [60]:
print ('shape of b : ', b)

shape of b :  tf.Tensor([2.6439803], shape=(1,), dtype=float32)


### Model Prediction on 1st Examples in Test Dataset

In [61]:
pred=prediction(X_test,W,b)
pred

<tf.Tensor: id=16593, shape=(200, 1), dtype=float32, numpy=
array([[5.287775 ],
       [5.2875805],
       [5.287441 ],
       [5.2872868],
       [5.2873816],
       [5.2880764],
       [5.287963 ],
       [5.2875824],
       [5.287302 ],
       [5.2872744],
       [5.287551 ],
       [5.2878036],
       [5.2879677],
       [5.2871723],
       [5.2877884],
       [5.287678 ],
       [5.2878547],
       [5.28751  ],
       [5.287779 ],
       [5.287405 ],
       [5.287567 ],
       [5.2871876],
       [5.287155 ],
       [5.28728  ],
       [5.287411 ],
       [5.287141 ],
       [5.287696 ],
       [5.28675  ],
       [5.286813 ],
       [5.2871933],
       [5.2879047],
       [5.287507 ],
       [5.2879295],
       [5.2877893],
       [5.287151 ],
       [5.2876463],
       [5.287258 ],
       [5.287209 ],
       [5.287731 ],
       [5.2872295],
       [5.287771 ],
       [5.2864695],
       [5.287985 ],
       [5.287569 ],
       [5.2872295],
       [5.2872   ],
       [5.287592 ],


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [21]:
iris = pd.read_csv("C:/Users/Parikshit/Desktop/great lakes course/Week 6 Assignment/Internal lab/11_Iris.csv")

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [22]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()
# apply le on categorical feature columns
iris.Species = le.fit_transform(iris.Species)

In [23]:
y_iris = tf.keras.utils.to_categorical(iris.Species, num_classes=3)

### Splitting the data into feature set and target set

In [24]:
X_iris = iris.iloc[:,1:5]
#y_iris = iris.Species
y_iris.shape

(150, 3)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [ ]:
# Initialize Sequential model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(4, input_dim=4, kernel_initializer='normal', activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [ ]:

# Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

# Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X_iris, y_iris, test_size=0.20,random_state=3)

### Model Training 

In [69]:
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=30)

W1202 19:50:14.527455  3120 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Train on 120 samples, validate on 30 samples
Epoch 1/30
120/120 [==============================] - 0s 2ms/sample - loss: 1.0969 - accuracy: 0.3333 - val_loss: 1.0682 - val_accuracy: 0.3333
Epoch 2/30
120/120 [==============================] - 0s 270us/sample - loss: 1.0409 - accuracy: 0.3333 - val_loss: 0.9929 - val_accuracy: 0.3333
Epoch 3/30
120/120 [==============================] - 0s 207us/sample - loss: 0.9829 - accuracy: 0.3583 - val_loss: 0.9743 - val_accuracy: 0.6667
Epoch 4/30
120/120 [==============================] - 0s 202us/sample - loss: 0.9458 - accuracy: 0.5583 - val_loss: 0.9194 - val_accuracy: 0.6333
Epoch 5/30
120/120 [==============================] - 0s 255us/sample - loss: 0.9179 - accuracy: 0.6000 - val_loss: 0.9013 - val_accuracy: 0.6333
Epoch 6/30
120/120 [==============================] - 0s 287us/sample - loss: 0.9049 - accuracy: 0.6083 - val_loss: 0.8813 - val_accuracy: 0.6667
Epoch 7/30
120/120 [==============================] - 0s 288us/sample - loss: 0.8

### Model Prediction

In [70]:
model.predict(testX)

W1202 19:50:16.432182  3120 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


array([[0.5057226 , 0.3004017 , 0.19387567],
       [0.5057226 , 0.3004017 , 0.19387567],
       [0.5057226 , 0.3004017 , 0.19387567],
       [0.5057226 , 0.3004017 , 0.19387567],
       [0.5057226 , 0.3004017 , 0.19387567],
       [0.00362541, 0.30818567, 0.6881889 ],
       [0.06420951, 0.42890084, 0.50688964],
       [0.5057226 , 0.3004017 , 0.19387567],
       [0.01516112, 0.37305996, 0.611779  ],
       [0.05910607, 0.42685047, 0.51404345],
       [0.07047994, 0.43091762, 0.49860248],
       [0.5057226 , 0.3004017 , 0.19387567],
       [0.07070117, 0.43141055, 0.4978883 ],
       [0.05990533, 0.42717877, 0.51291597],
       [0.01327026, 0.36621106, 0.6205187 ],
       [0.5057226 , 0.3004017 , 0.19387567],
       [0.02828279, 0.4007393 , 0.57097787],
       [0.00906469, 0.34946483, 0.64147043],
       [0.02006944, 0.3851078 , 0.59482276],
       [0.5057226 , 0.3004017 , 0.19387567],
       [0.03044245, 0.40306965, 0.56648797],
       [0.01599538, 0.37518153, 0.60882306],
       [0.

### Save the Model

In [71]:
#import pickle
#filename = 'finalized_model.sav'
#pickle.dump(model, open(filename, 'wb'))

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

"""
# how to load model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
"""

Saved model to disk


'\n# how to load model\njson_file = open(\'model.json\', \'r\')\nloaded_model_json = json_file.read()\njson_file.close()\nloaded_model = model_from_json(loaded_model_json)\n# load weights into new model\nloaded_model.load_weights("model.h5")\nprint("Loaded model from disk")\n'

In [72]:
# serialize model to YAML
model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
"""
# How to load model from yaml
# load YAML and create model
yaml_file = open('model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
"""




Saved model to disk


'\n# How to load model from yaml\n# load YAML and create model\nyaml_file = open(\'model.yaml\', \'r\')\nloaded_model_yaml = yaml_file.read()\nyaml_file.close()\nloaded_model = model_from_yaml(loaded_model_yaml)\n# load weights into new model\nloaded_model.load_weights("model.h5")\nprint("Loaded model from disk")\n'

In [73]:
# Save model in keras
model.save("model.h5")
print("Saved model to disk")

# how to load model from keras 
"""
# load model
model = load_model('model.h5')
# summarize model.
model.summary()
"""

Saved model to disk


"\n# load model\nmodel = load_model('model.h5')\n# summarize model.\nmodel.summary()\n"

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?

In [1]:
# Initialize Sequential model
model_layer = tf.keras.models.Sequential()
model_layer.add(tf.keras.layers.Dense(4, input_dim=4, kernel_initializer='normal', activation='relu'))
model_layer.add(tf.keras.layers.Dense(3, input_dim=3, kernel_initializer='normal', activation='relu'))
model_layer.add(tf.keras.layers.Dense(3, input_dim=3, kernel_initializer='normal', activation='relu'))
model_layer.add(tf.keras.layers.Dense(3, activation='softmax'))

NameError: name 'tf' is not defined